In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Conv1D, MaxPooling1D
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam
from tqdm import tqdm

In [2]:
# Set the path to the audio dataset
data_dir = "time"

# Read metadata
metadata_path = "original.csv"
metadata = pd.read_csv(metadata_path)

# Extract audio features using librosa
audio_data = []
labels = []
for index, row in tqdm(metadata.iterrows()):
    audio_file = os.path.join(data_dir, row["filename"])
    signal, sr = librosa.load(audio_file, sr=None)
    mfcc_features =librosa.feature.mfcc(y=signal ,sr=sr,n_mfcc=40)
    features = np.mean(mfcc_features.T,axis=0)
    audio_data.append(features)
    labels.append(row["tb_status"])

# Convert the lists to numpy arrays
audio_data = np.array(audio_data)
labels = np.array(labels)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    audio_data, labels, test_size=0.2, random_state=42
)


27368it [02:04, 219.34it/s]


In [3]:
timesteps = audio_data.shape[1]
features = audio_data.shape[2] if len(audio_data.shape) > 2 else audio_data.shape[1]

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(40, 1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))


# Compile the model
model.compile(loss="binary_crossentropy", optimizer = Adam(learning_rate=0.0001), metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32,validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



Epoch 1/20
616/616 [==============================] - 19s 25ms/step - loss: 0.6720 - accuracy: 0.5778 - val_loss: 0.6568 - val_accuracy: 0.6027
Epoch 2/20
616/616 [==============================] - 15s 24ms/step - loss: 0.6318 - accuracy: 0.6467 - val_loss: 0.6336 - val_accuracy: 0.6466
Epoch 3/20
616/616 [==============================] - 15s 24ms/step - loss: 0.6048 - accuracy: 0.6731 - val_loss: 0.5986 - val_accuracy: 0.6817
Epoch 4/20
616/616 [==============================] - 15s 24ms/step - loss: 0.5871 - accuracy: 0.6855 - val_loss: 0.5915 - val_accuracy: 0.6753
Epoch 5/20
616/616 [==============================] - 15s 24ms/step - loss: 0.5779 - accuracy: 0.6902 - val_loss: 0.5841 - val_accuracy: 0.6776
Epoch 6/20
616/616 [==============================] - 15s 25ms/step - loss: 0.5706 - accuracy: 0.6973 - val_loss: 0.5749 - val_accuracy: 0.6950
Epoch 7/20
616/616 [==============================] - 16s 26ms/step - loss: 0.5658 - accuracy: 0.6983 - val_loss: 0.5733 - val_accuracy:

In [4]:
# model.save('lstm_model_60')

In [ ]:
timesteps = audio_data.shape[1]
features = audio_data.shape[2] if len(audio_data.shape) > 2 else audio_data.shape[1]

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(40, 1)))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))


# Compile the model
model.compile(loss="binary_crossentropy", optimizer = Adam(learning_rate=0.0001), metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_train, y_train)
print("Train Loss:", loss)
print("Train Accuracy:", accuracy)
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



Epoch 1/100
548/548 [==============================] - 18s 27ms/step - loss: 0.6778 - accuracy: 0.5668 - val_loss: 0.6654 - val_accuracy: 0.5969
Epoch 2/100
548/548 [==============================] - 13s 25ms/step - loss: 0.6326 - accuracy: 0.6470 - val_loss: 0.6266 - val_accuracy: 0.6497
Epoch 3/100
548/548 [==============================] - 13s 24ms/step - loss: 0.5999 - accuracy: 0.6820 - val_loss: 0.5918 - val_accuracy: 0.6755
Epoch 4/100
548/548 [==============================] - 12s 21ms/step - loss: 0.5786 - accuracy: 0.6958 - val_loss: 0.5775 - val_accuracy: 0.6839
Epoch 5/100
548/548 [==============================] - 11s 21ms/step - loss: 0.5706 - accuracy: 0.6969 - val_loss: 0.5754 - val_accuracy: 0.6878
Epoch 6/100
548/548 [==============================] - 15s 27ms/step - loss: 0.5658 - accuracy: 0.7010 - val_loss: 0.5685 - val_accuracy: 0.6926
Epoch 7/100
548/548 [==============================] - 14s 26ms/step - loss: 0.5614 - accuracy: 0.7043 - val_loss: 0.5668 - val_ac

548/548 [==============================] - 7s 12ms/step - loss: 0.4450 - accuracy: 0.7848 - val_loss: 0.4972 - val_accuracy: 0.7509
Epoch 58/100
548/548 [==============================] - 8s 15ms/step - loss: 0.4445 - accuracy: 0.7828 - val_loss: 0.5086 - val_accuracy: 0.7470
Epoch 59/100
548/548 [==============================] - 14s 25ms/step - loss: 0.4432 - accuracy: 0.7855 - val_loss: 0.5185 - val_accuracy: 0.7433
Epoch 60/100
548/548 [==============================] - 14s 26ms/step - loss: 0.4383 - accuracy: 0.7895 - val_loss: 0.5035 - val_accuracy: 0.7545
Epoch 61/100
548/548 [==============================] - 12s 21ms/step - loss: 0.4361 - accuracy: 0.7880 - val_loss: 0.4967 - val_accuracy: 0.7547
Epoch 62/100
548/548 [==============================] - 7s 13ms/step - loss: 0.4350 - accuracy: 0.7892 - val_loss: 0.5027 - val_accuracy: 0.7509
Epoch 63/100
548/548 [==============================] - 7s 12ms/step - loss: 0.4296 - accuracy: 0.7946 - val_loss: 0.5122 - val_accuracy: 0.